In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

#open window
driver = webdriver.Chrome()

driver.get("https://openai-openai-detector.hf.space/")
driver.maximize_window()

#locate elements
elem = driver.find_element(By.ID, "textbox")

#imp functions

# class for custom condition for Webdriverwait
class text_matches_float_regex:
    def __init__(self, locator):
        self.locator = locator
        self.pattern = re.compile(r'^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$')

    def __call__(self, driver):
        element = driver.find_element(*self.locator)

        if self.pattern.match(element.text) is not None :
            return element
        else:
            return False

# function for removing html tags
def remove_html_tags(text):
    clean = re.compile('<.*?>')  # Compile the regex pattern for HTML tags
     # Remove HTML tags
    text = re.sub(clean, '', text)
    # Replace underscores with empty spaces
    text = text.replace('_', ' ')
    return text

# Main scrapping function
def ai_text_score(text):
    elem.clear()
    elem.send_keys(text)
    time.sleep(5)
    # if error.is_displayed():
    #     return ai_text_score(text)
    pred = driver.find_element(By.ID, "message")
    while pred.text == "Predicting ...":
        time.sleep(5)
    res = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "fake-percentage")))
    print(res)
    return res.text

#load dataframe and clean it
df = pd.read_excel('./Results.xlsx')
df = df.dropna(axis = 0 , ignore_index=False , subset= 'Abstract')
df['Abstract'] = df["Abstract"].apply(remove_html_tags)

last_index = 700 #last index till the loop will run

try:
    for ind in df.index:
        if pd.isnull(df['GPT-2'][ind]):
            print("runs")
            df['GPT-2'][ind]= ai_text_score(df['Abstract'][ind])
        if ind == last_index:
            break
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
     driver.close()
df.to_excel("Results.xlsx", index = False)
print(df)